<a href="https://colab.research.google.com/github/pedropalmav/proyecto-recsys/blob/main/MistralLora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-12-09 15:52:32--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uccb9b950753b137eb60017ac274.dl.dropboxusercontent.com/cd/0/inline/Cf5djvwF80rutQNObzlQ14aXOoosWBdt0QSnrdhV2hiTQziN4T5cXX3eDZC3Z0Yxg_EVHWuy2zP9Boqy26BtVkT-YAhfvS3zUsd_S6kuAu5iGmAvuj7XRnL5ADnUY324_iw/file?dl=1# [following]
--2024-12-09 15:52:33--  https://uccb9b950753b137eb60017ac274.dl.dropboxusercontent.com/cd/0/inline/Cf5djvwF80rutQNObzlQ14aXOoosWBdt0QSnrdhV2hiTQziN4T5cXX3eDZC3Z0Yxg_EVHWuy2zP9Boqy26BtVkT-YAhfvS3zUsd_S6kuAu5iGmAvuj7XRnL5ADnUY324_iw/file?dl=1
Resolving uccb9b950753b137eb60017ac274.dl.dropboxusercontent.com (uccb9b950753b137eb60017ac274.dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:601d:15::a27d

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
#model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [8]:
def generate_prompt(data):
  question = data['question']
  ref = data['references']

  chat = [
      {"role": "user", "content": f"Answer the Question: {question} Considering this information: {ref}"},
    ]

  data['prompt'] = tokenizer.apply_chat_template(chat, tokenize=False)
  return data

#dataset['test'] = dataset['test'].map(generate_prompt)

In [9]:
def generate_zeroshot_prompt(likes, dislikes):
  likes_str = "\n".join(likes)
  dislikes_str = "\n".join(dislikes)

  chat = [
    {"role": "user", "content": f"I've liked these movies: {likes_str}. I've disliked these movies: {dislikes_str}. Recommend a new movie for me to watch."}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False)

In [10]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = (
        f"Pretend you are a movie recommender system. I will give you a conversation between a human and assistant. Based on the conversation, you reply me with 10 recommendations, including only their titles without extra sentences or description. Here is the conversation: {conversation}.\n"
        f"\n"
    )
    messages = [
        {"role": "user", "content": content}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    return prompt

In [11]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [12]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [13]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

## DATASET

In [14]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.1

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [15]:
dataset = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  for conv_number, conversation in enumerate(user_data['Conversation']):
    conversation_id = conversation[f"conversation_{conv_number+1}"]["conversation_id"]
    prompt = generate_zeroshot(conversation_id, conversations)
    dataset.append({
        "user_id": user_id,
        "conversation_id": conversation_id,
        "user_might_like": user_data['user_might_like'],
        "prompt": prompt
    })

In [16]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
print("Train prompts:", len(train_set))
print("Test prompts:", len(test_set))

Train prompts: 680
Test prompts: 171


In [17]:
conversations_id = []
conversations_train = []
for data in train_set:
  conversations_id.append(data['conversation_id'])

for conversation in conversations:
  if conversation['conversation_id'] in conversations_id:
    conversations_train.append(conversation)

## Lora

In [18]:
import os
import peft

#os.environ["CUDA_VISIBLE_DEVICES"] = "0" # si solo vamos a usar la primera GPU que tengamos

import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoConfig, GPTJForCausalLM
from peft import LoraConfig, get_peft_model

In [19]:
# Settings for A100 - For 3090
MICRO_BATCH_SIZE = 4  # change to 8 for A100
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # paper uses 3
LEARNING_RATE = 2e-5
CUTOFF_LEN = 256
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [20]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [21]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

dataset = Dataset.from_list(conversations_train)

def flatten_and_tokenize_function(examples):
    combined_texts = []
    for messages in examples['messages']:
        flattened_messages = []
        for message in messages:
            if isinstance(message, list):
                flattened_messages.extend(message)
            else:
                flattened_messages.append(message)
        combined_texts.append(" ".join(flattened_messages))

    return tokenizer(
        combined_texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(flatten_and_tokenize_function, batched=True, remove_columns=["conversation_id", "messages"])

training_args = TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=1,
    output_dir="./result",
    save_total_limit=3,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.233900
2,2.251600
3,2.236700
4,2.227800
5,2.195600
6,2.238000
7,2.231000
8,2.216100
9,2.231100
10,2.216900


TrainOutput(global_step=10, training_loss=2.2278627157211304, metrics={'train_runtime': 520.0886, 'train_samples_per_second': 2.615, 'train_steps_per_second': 0.019, 'total_flos': 6994830714470400.0, 'train_loss': 2.2278627157211304, 'epoch': 1.8941176470588235})

In [22]:
trainer.save_model("Mistral-7B-Instruct.h5")

In [24]:
#mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

base_model = "mistralai/Mistral-7B-Instruct-v0.3"
adapter_model = "Mistral-7B-Instruct.h5"

model = AutoModelForCausalLM.from_pretrained(base_model)
model = PeftModel.from_pretrained(model, adapter_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCa

In [27]:
from tqdm.auto import tqdm

In [28]:
for i in tqdm(range(len(test_set))):
  item = test_set[i]
  result = text_generator(item['prompt'], max_new_tokens=200, return_full_text=False)
  item['response'] = result[0]['generated_text']

  0%|          | 0/171 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Settin

In [29]:
answers = []
for item in test_set:
  answers.append(item['response'])

In [30]:
print(len(answers))

171


# NDCG & Recall

In [31]:
import re

def parse_recommendations(answer):
  pattern = r'\d+\.\s+"*([^"]+?)"*(?:\s)?(?:\s*-\s*.+)?(?:\s*)?$'
  recommendations = re.findall(pattern, answer, re.MULTILINE)
  return recommendations

In [32]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [33]:
def evaluate_model(answers, users):
    assert len(answers) == len(users)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 10:
         continue
       user_data = read_user_data(path_movie_data, users[i]["user_id"])
       user_might_like = [item_map[id] for id in user_data["user_might_like"]]
       print(user_might_like)
       print(recommendations)
       rel_vector = np.isin(recommendations, user_might_like, assume_unique=True).astype(int)

       mean_ndcg += ndcg_at_k(rel_vector, 20, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 20, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [34]:
ndcg, recall = evaluate_model(answers, test_set)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

['Conspiracy', 'Heat', 'Heat', 'NYPD Blue - Season 2', 'NYPD Blue - The Complete Third Season']
['Saving Private Ryan (1998)', 'Band of Brothers (2001)', 'We Were Soldiers (2002)', 'Letters from Iwo Jima (2006)', 'The Pacific (2010)', 'Fury (2014)', 'American Sniper (2014)', 'Hacksaw Ridge (2016)', 'The Imitation Game (2014)', "Schindler's List (1993)"]
['The Lord Of The Rings: The Return Of The King', 'Girl with a Pearl Earring', 'Collateral']
['Rush', 'Le Mans', 'Grand Prix', 'Senna', 'Ford v Ferrari', 'The French Connection', 'Inception', 'The Bourne Identity', 'The Italian Job', 'The Thomas Crown Affair']
["Pan's Labyrinth - Limited Edition Mondo X Steelbook [Blu-ray - DVD]", "Pan's Labyrinth [Blu-ray] [Blu-ray] (2009)", "The Devil's Backbone", 'Battleship Potemkin VHS', 'Fallen Idol VHS', 'Black Snake Moan [Theatrical Release]', 'Warlock', 'No Country For Old Men', 'The Bridge Die Brucke  VHS', 'A Double Life', '3:10 to Yuma [Theatrical Release]']
['Double Indemnity', 'Mildred Pie

In [35]:
ndcg

0.014344407936064509

In [36]:
recall

0.01297939150051826